In [5]:
import pandas as pd

df = pd.read_csv("goodreads_library_export.csv")

In [6]:
df["Date Read"] = pd.to_datetime(df["Date Read"], errors="coerce").dt.normalize()
df["Date Added"] = pd.to_datetime(df["Date Added"], errors="coerce").dt.normalize()

subset = df[[
    "Title", "Author", "My Rating", 
    "Average Rating", "Number of Pages", 
    "Year Published", "Date Read", 
    "Date Added", "Exclusive Shelf", 
    "Read Count"]]

books_2025 = subset[
    (subset["Exclusive Shelf"] == "read") &
    (
        subset["Date Read"].between("2025-01-01", "2025-12-31") |
        (
            subset["Date Read"].isna() &
            subset["Date Added"].between("2025-01-01", "2025-12-31")
        )
    )
]
books_2025


,Title,Author,My Rating,Average Rating,Number of Pages,Year Published,Date Read,Date Added,Exclusive Shelf,Read Count
4,The First Bad Man,Miranda July,4,3.67,288.0,2015.0,NaT,2025-11-24,read,1
9,Jane and Dan at the End of the World,Colleen Oakley,3,3.68,368.0,2025.0,2025-11-16,2025-09-16,read,1
12,"Sister Wife: A Memoir of Faith, Family, and Fi...",Christine Brown Woolley,4,4.23,320.0,2025.0,2025-11-03,2025-09-08,read,1
14,Vera Wong's Unsolicited Advice for Murderers (...,Jesse Q. Sutanto,4,4.02,339.0,2023.0,2025-10-24,2025-10-13,read,1
24,Penance,Eliza Clark,5,3.86,336.0,2023.0,2025-10-12,2025-10-01,read,1
27,Colored Television,Danzy Senna,3,3.54,277.0,2024.0,2025-10-01,2025-06-22,read,1
30,Nine Perfect Strangers,Liane Moriarty,5,3.56,453.0,2018.0,2025-09-24,2025-09-19,read,1
32,The Compound,Aisling Rawle,3,3.57,292.0,2025.0,2025-09-16,2025-09-11,read,1
34,Here One Moment,Liane Moriarty,5,3.99,512.0,2024.0,2025-09-10,2025-09-01,read,1
36,What Alice Forgot,Liane Moriarty,4,4.07,487.0,2018.0,2025-08-29,2025-08-04,read,1


In [11]:

#Monthly Stats
januaryBooks = books_2025[
    (books_2025["Month Number"] == 1)
]

newStats = {
    'Months': ['January', 'February', 'March', 
               'April', 'May', 'June', 
               'July', 'August', 'September', 
               'October', 'November', 'December'],
}

bookStats = pd.DataFrame(newStats)
# len(books_2025[(books_2025["Month Number"] == 1)])

booksPerMonth = books_2025["Month Name"].value_counts()
bookStats["Books Read"] = bookStats["Months"].map(booksPerMonth).fillna(0).astype(int)


avgRating = books_2025.groupby("Month Name")["My Rating"].sum() / booksPerMonth
bookStats["Your Average Rating"] = bookStats["Months"].map(avgRating).fillna(0).astype(float)

globalRating = books_2025.groupby("Month Name")["Average Rating"].sum() / booksPerMonth
bookStats["Global Average Rating"] = bookStats["Months"].map(globalRating).fillna(0).astype(float)

#Idea, add if your rating tended to be higher or lower than global


pagesRead = books_2025.groupby("Month Name")["Number of Pages"].sum()
bookStats["Pages Read"] = bookStats["Months"].map(pagesRead).fillna(0).astype(int)

fiveStarBooks = books_2025[books_2025["My Rating"] == 5]
fiveStarCounts = fiveStarBooks.groupby("Month Name").size()

bookStats["Five Star Reviews"] = bookStats["Months"].map(fiveStarCounts).fillna(0).astype(int)

percentageOfFiveStar = (bookStats["Five Star Reviews"]/bookStats["Books Read"]) * 100
bookStats["Percentage of Five Star Books"] = percentageOfFiveStar.fillna(0).astype(float)

minPagesIndex = books_2025["Number of Pages"].idxmin()
minPagesBook = books_2025.loc[minPagesIndex]



maxPagesIndex = books_2025["Number of Pages"].idxmax()
maxPagesBook = books_2025.loc[maxPagesIndex]

maxPagesBook


#Idea: Best month of reading, month with highest percentage of five star books. Worst month of reading would be harder to decide i think

Title                     Here One Moment
Author                     Liane Moriarty
My Rating                               5
Average Rating                       3.99
Number of Pages                     512.0
Year Published                     2024.0
Date Read             2025-09-10 00:00:00
Date Added            2025-09-01 00:00:00
Exclusive Shelf                      read
Read Count                              1
Official Date Read    2025-09-10 00:00:00
Month Number                            9
Month Name                      September
Name: 34, dtype: object

In [9]:
# ============== STATS ==============
books_2025 = books_2025.copy()

YEARtotalBooks = len(books_2025)

YEARpagesRead = int(books_2025["Number of Pages"].sum())

YEARyourAvgRating = (books_2025["My Rating"].sum()) / YEARtotalBooks

YEARfiveStar = books_2025[books_2025["My Rating"] == 5]
YEARnumFiveStar = len(YEARfiveStar)
YEARpercentFiveStar = (YEARnumFiveStar / YEARtotalBooks) * 100

books_2025["Official Date Read"] = books_2025["Date Read"].fillna(books_2025["Date Added"])
books_2025["Month Number"] = books_2025["Official Date Read"].dt.month
books_2025["Month Name"] = books_2025["Official Date Read"].dt.month_name()



In [12]:
bookStats

,Months,Books Read,Your Average Rating,Global Average Rating,Pages Read,Five Star Reviews,Percentage of Five Star Books
0,January,7,2.857143,3.927143,2567,1,14.285714
1,February,3,3.666667,4.166667,992,0,0.000000
2,March,4,3.750000,3.725000,1328,0,0.000000
3,April,7,4.000000,3.882857,2396,2,28.571429
4,May,9,3.888889,3.812222,3134,3,33.333333
5,June,6,4.333333,3.896667,2099,3,50.000000
6,July,3,2.333333,3.826667,956,1,33.333333
7,August,3,3.666667,3.750000,1063,0,0.000000
8,September,3,4.333333,3.706667,1257,2,66.666667
9,October,3,4.000000,3.806667,952,1,33.333333


In [ ]:
# ============= REPORT
print("Total number of books read in 2025:", YEARtotalBooks)

print("\nMonthly Breakdown of books read:")
print(bookStats[["Months", "Books Read"]])

print("\nTotal number of pages read in 2025:", YEARpagesRead)
print("\n Monthly breakdown of pages read:")
print(bookStats[["Months", "Pages Read"]])

print(f"\nYour average rating: {YEARyourAvgRating:.2f}")
print("You rated", YEARnumFiveStar, "books five stars")
print(f"That is {YEARpercentFiveStar:.2f} % of your total books read")

print("\nThe longest book you read was", maxPagesBook["Title"], "by", maxPagesBook["Author"])
print("The shortest book you read was", minPagesBook["Title"], "by", minPagesBook["Author"])



In [19]:
books_2025.head()

,Title,Author,My Rating,Average Rating,Number of Pages,Year Published,Date Read,Date Added,Exclusive Shelf,Read Count,Official Date Read,Month Number,Month Name
4,The First Bad Man,Miranda July,4,3.67,288.0,2015.0,NaT,2025-11-24,read,1,2025-11-24,11,November
9,Jane and Dan at the End of the World,Colleen Oakley,3,3.68,368.0,2025.0,2025-11-16,2025-09-16,read,1,2025-11-16,11,November
12,"Sister Wife: A Memoir of Faith, Family, and Fi...",Christine Brown Woolley,4,4.23,320.0,2025.0,2025-11-03,2025-09-08,read,1,2025-11-03,11,November
14,Vera Wong's Unsolicited Advice for Murderers (...,Jesse Q. Sutanto,4,4.02,339.0,2023.0,2025-10-24,2025-10-13,read,1,2025-10-24,10,October
24,Penance,Eliza Clark,5,3.86,336.0,2023.0,2025-10-12,2025-10-01,read,1,2025-10-12,10,October


In [23]:
books_2025['Date Read'] = pd.to_datetime(books_2025['Date Read'])
books_2025['Date Added'] = pd.to_datetime(books_2025['Date Added'])
books_2025['Time to Read'] = books_2025['Date Read'] - books_2025['Date Added']
books_2025['Time to Read days'] = books_2025['Time to Read'].dt.days

books_2025

,Title,Author,My Rating,Average Rating,Number of Pages,Year Published,Date Read,Date Added,Exclusive Shelf,Read Count,Official Date Read,Month Number,Month Name,Time to Read,Time to Read days
4,The First Bad Man,Miranda July,4,3.67,288.0,2015.0,NaT,2025-11-24,read,1,2025-11-24,11,November,NaT,NaN
9,Jane and Dan at the End of the World,Colleen Oakley,3,3.68,368.0,2025.0,2025-11-16,2025-09-16,read,1,2025-11-16,11,November,61 days,61.0
12,"Sister Wife: A Memoir of Faith, Family, and Fi...",Christine Brown Woolley,4,4.23,320.0,2025.0,2025-11-03,2025-09-08,read,1,2025-11-03,11,November,56 days,56.0
14,Vera Wong's Unsolicited Advice for Murderers (...,Jesse Q. Sutanto,4,4.02,339.0,2023.0,2025-10-24,2025-10-13,read,1,2025-10-24,10,October,11 days,11.0
24,Penance,Eliza Clark,5,3.86,336.0,2023.0,2025-10-12,2025-10-01,read,1,2025-10-12,10,October,11 days,11.0
27,Colored Television,Danzy Senna,3,3.54,277.0,2024.0,2025-10-01,2025-06-22,read,1,2025-10-01,10,October,101 days,101.0
30,Nine Perfect Strangers,Liane Moriarty,5,3.56,453.0,2018.0,2025-09-24,2025-09-19,read,1,2025-09-24,9,September,5 days,5.0
32,The Compound,Aisling Rawle,3,3.57,292.0,2025.0,2025-09-16,2025-09-11,read,1,2025-09-16,9,September,5 days,5.0
34,Here One Moment,Liane Moriarty,5,3.99,512.0,2024.0,2025-09-10,2025-09-01,read,1,2025-09-10,9,September,9 days,9.0
36,What Alice Forgot,Liane Moriarty,4,4.07,487.0,2018.0,2025-08-29,2025-08-04,read,1,2025-08-29,8,August,25 days,25.0
